In [2]:
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
import pandas as pd

In [3]:
# Turn this into a function that returns the reporting_client

#Create the credentials object
service_credentials_file = 'client_secrets.json'
scopes = ['https://www.googleapis.com/auth/analytics.readonly','https://www.googleapis.com/auth/analytics.edit']
credentials = ServiceAccountCredentials.from_json_keyfile_name(service_credentials_file, scopes=scopes)

#Build a service object
api_name = 'analyticsreporting'
api_version = 'v4'
reporting_client = build(api_name, api_version, credentials=credentials)

In [4]:
# Turn into a report getting function

view_id = '95100800'

report_output = reporting_client.reports().batchGet(
    body={
    'reportRequests': [
        {
          'viewId': view_id,
          'dateRanges': [{'startDate': '28daysAgo', 'endDate': 'today'}],
          'metrics': [{'expression': 'ga:sessions'}],
          'dimensions': [{'name': 'ga:country'}]
        }]
      }
    ).execute()

In [5]:
type(report_output)

dict

In [6]:
report_output['reports']

[{'columnHeader': {'dimensions': ['ga:country'],
   'metricHeader': {'metricHeaderEntries': [{'name': 'ga:sessions',
      'type': 'INTEGER'}]}},
  'data': {'maximums': [{'values': ['1']}],
   'minimums': [{'values': ['1']}],
   'rowCount': 2,
   'rows': [{'dimensions': ['Italy'], 'metrics': [{'values': ['1']}]},
    {'dimensions': ['Philippines'], 'metrics': [{'values': ['1']}]}],
   'totals': [{'values': ['2']}]}}]

In [ ]:
pd.r

In [11]:
# Will need to modify this for my needs, temporarily borrowed
#Parse the response of API
def parse_response(report):

    """Parses and prints the Analytics Reporting API V4 response"""
    #Initialize results, in list format because two dataframes might return
    result_list = []

    #Initialize empty data container for the two dateranges (if there are two that is)
    data_csv = []
    data_csv2 = []

    #Initialize header rows
    header_row = []

    #Get column headers, metric headers, and dimension headers.
    columnHeader = report.get('columnHeader', {})
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
    dimensionHeaders = columnHeader.get('dimensions', [])

    #Combine all of those headers into the header_row, which is in a list format
    for dheader in dimensionHeaders:
        header_row.append(dheader)
    for mheader in metricHeaders:
        header_row.append(mheader['name'])

    #Get data from each of the rows, and append them into a list
    rows = report.get('data', {}).get('rows', [])
    for row in rows:
        row_temp = []
        dimensions = row.get('dimensions', [])
        metrics = row.get('metrics', [])
        for d in dimensions:
            row_temp.append(d)
        for m in metrics[0]['values']:
            row_temp.append(m)
            data_csv.append(row_temp)

        #In case of a second date range, do the same thing for the second request
        if len(metrics) == 2:
            row_temp2 = []
            for d in dimensions:
                row_temp2.append(d)
            for m in metrics[1]['values']:
                row_temp2.append(m)
            data_csv2.append(row_temp2)

    #Putting those list formats into pandas dataframe, and append them into the final result
    result_df = pd.DataFrame(data_csv, columns=header_row)
    result_list.append(result_df)
    if data_csv2 != []:
        result_list.append(pandas.DataFrame(data_csv2, columns=header_row))

    return result_list

In [12]:
result = parse_response(report_output)

In [14]:
type(result)

list